# Extending `pandas` to other ecosystems

In [1]:
    import ibis

    db = ibis.sqlite.connect('idiomatic_pandas.sqlite')

    db.list_tables()
    db.table('urls')['value'].execute()
    db.table('responses')['key'].execute()